<a href="https://colab.research.google.com/github/hyunaeee/upstage/blob/main/GPT_last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydantic-settings
!pip install pathlib
!pip install rouge

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [ ]:
from pydantic_settings import BaseSettings
import pandas as pd
import openai
from openai import OpenAI
import json
from pathlib import Path
from rouge import Rouge


In [ ]:

class Settings(BaseSettings):
    openai_api_key: str

    class Config:
        env_file = ".env"
        env_file_encoding = "utf-8"

settings = Settings()

In [ ]:
# OpenAI API 키 설정
openai.api_key = settings.openai_api_key

# 데이터 로드
train_file = '/data/ephemeral/home/data/train.csv'
dev_file = '/data/ephemeral/home/data/dev.csv'
submission_file = '/data/ephemeral/home/data/sample_submission.csv'
test_file = '/data/ephemeral/home/data/test.csv'

train_df = pd.read_csv(train_file)
dev_df = pd.read_csv(dev_file)
submission_df = pd.read_csv(submission_file)
test_df = pd.read_csv(test_file)

# 'fname' 기준으로 'dialogue' 매핑
submission_df = submission_df.merge(test_df[['fname', 'dialogue']], on='fname', how='left')

In [ ]:
submission_df = pd.read_csv(submission_file)
submission_df = submission_df.merge(test_df[['fname', 'dialogue']], on='fname', how='left')

In [ ]:
def convert_to_chat_format(df, output_path):
    with open(output_path, 'w') as f:
        for _, row in df.iterrows():
            json.dump({
                "messages": [
                    {"role": "system", "content": "You summarize dialogues in a formal written style with a slight awkwardness, mimicking English-to-Korean translation, while capturing all key details."},
                    {"role": "user", "content": row['dialogue']},
                    {"role": "assistant", "content": row['summary']}
                ]
            }, f)
            f.write('\n')

In [ ]:
convert_to_chat_format(train_df, '/data/ephemeral/home/data/train.jsonl')
convert_to_chat_format(dev_df, '/data/ephemeral/home/data/dev.jsonl')

In [ ]:
client = OpenAI(
    api_key = settings.openai_api_key
)

train_data_file = client.files.create(
    file=Path("/data/ephemeral/home/data/train.jsonl"),
    purpose="fine-tune",
)
validation_data_file = client.files.create(
    file=Path("/data/ephemeral/home/data/dev.jsonl"),
    purpose="fine-tune",
)

print(f"File ID: {train_data_file.id}")
print(f"File ID: {validation_data_file.id}")

File ID: file-HQcAjWtiaygkk52hF2cyxn
File ID: file-LV7wLFLMTowV8MVvLTncQj


In [ ]:
def fine_tune(train_file_id, validation_file_id):
    try:
        # 파인 튜닝 작업을 생성하고 결과를 저장합니다.
        fine_tune_job = client.fine_tuning.jobs.create(
            model="gpt-3.5-turbo",
            training_file=train_file_id,
            validation_file = validation_file_id
        )

        # 생성된 파인 튜닝 작업의 ID를 사용하여 상태를 검색합니다.
        job_id = fine_tune_job.id  # 작업 ID를 얻습니다.
        print(f"Fine-tune job ID: {job_id}")  # 작업 ID를 출력합니다.

        # 작업 ID를 사용하여 파인 튜닝 작업의 상태를 검색합니다.
        job_status = client.fine_tuning.jobs.retrieve(job_id)
        print(job_status)  # 작업 상태를 출력합니다.

    except openai.APIConnectionError as e:
        print("The server could not be reached")
        print(e.__cause__)
    except openai.RateLimitError as e:
        print("A 429 status code was received; we should back off a bit.")
    except openai.APIStatusError as e:
        print("Another non-200-range status code was received")
        print(e.status_code)
        print(e.response)

In [ ]:
fine_tune(train_data_file.id, validation_data_file.id)

Fine-tune job ID: ftjob-LFJKHw7jhAZsYIWs9sSnbJYZ
FineTuningJob(id='ftjob-LFJKHw7jhAZsYIWs9sSnbJYZ', created_at=1732605145, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-U97IeTnyW6WqCetlCsDti76h', result_files=[], seed=761283474, status='validating_files', trained_tokens=None, training_file='file-YYoNTQNRv1DysSLa7cq9LL', validation_file='file-SfScyHqZ4dQWJsd5GE4vjC', estimated_finish=None, integrations=[], user_provided_suffix=None)


In [ ]:
fine_tuned_model = client.fine_tuning.jobs.retrieve("ftjob-LFJKHw7jhAZsYIWs9sSnbJYZ").fine_tuned_model
print(f"Fine-tuned model ID: {fine_tuned_model}")

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal::AXkfQa75


In [ ]:

def summarize_with_gpt(dialogue, index=None):
    try:
        if index is not None:
            print(f"Processing index: {index}")
        response = openai.chat.completions.create(
            model= fine_tuned_model,
            messages=[
                {"role": "system", "content": (
                    "You are a summarization assistant specializing in generating Korean summaries that feel like machine-translated text from English. "
                    "The summaries should retain an unnatural, translated style. Avoid overly fluent Korean sentences, and include slightly awkward expressions typical of translations. "
                    "Make sure to include key details from the dialogue."
                    "Make sure the summary is concise and does not exceed 21 words. Retain only the most important details while keeping a translated, unnatural tone."
                    "\n\nImportant Guidelines:\n"
                    "- Retain details like actions, reasons, and emotions mentioned in the dialogue.\n"
                    "- Ensure the summary feels like it was directly translated from English to Korean without polishing.\n"
                    "- The summary length should be proportional to the dialogue content, avoiding overly short summaries."
                    "\n\nHere are some examples of the desired style:\n"
                    "\nExample 1:"
                    "\nDialogue:\n"
                    "#Person1#: 주문하시겠습니까?"
                    "#Person2#: 아직 결정하지 못했습니다. 조금 더 시간을 주실 수 있나요?"
                    "#Person1#: 네, 천천히 결정하셔도 됩니다."
                    "#Person2#: 먼저 음료를 가져다 주실 수 있나요? 맥주 두 병을 원합니다."
                    "#Person1#: 알겠습니다."
                    "#Person2#: 오늘의 특선 메뉴를 알려주실 수 있나요?"
                    "#Person1#: 오늘의 특선 메뉴는 스테이크입니다."
                    "#Person2#: 그럼 이 스테이크 디너를 주문하겠습니다."
                    "#Person1#: 스테이크와 함께 드실 것은 무엇인가요?"
                    "#Person2#: 완두콩과 당근입니다."
                    "#Person1#: 알겠습니다. 디저트로 무엇을 드시겠습니까?"
                    "#Person2#: 아이스크림 부탁드립니다."
                    "\nSummary:\n"
                    "#Person1#은 #Person2#의 주문을 받아 맥주 두 병, 스테이크 디너, 그리고 아이스크림을 준비합니다."
                    "\nExample 2:"
                    "\nDialogue:\n"
                    "#Person1#: 요즘 모든 사람들이 다이어트를 하는 것 같아요. 그걸 눈치채셨나요?"
                    "#Person2#: 모두가 그 얘기를 하는 것 같아요, 특히 여자들이요."
                    "#Person1#: 제 친구가 고구마 다이어트를 했어요. 아침에 고구마를 먹고, 점심에 고구마를 먹고, 저녁에 또 고구마를 먹었어요."
                    "#Person2#: 그건 건강에 안 좋아요. 그는 체중을 줄였나요?"
                    "#Person1#: 네. 하지만 그는 오래 가지 못했어요. 어느 날 그는 아파서 병원에 실려갔어요. 의사가 그를 일주일 동안 병원에 머물게 했어요."
                    "#Person2#: 그런 일이 있었다니 정말 안타깝네요. 체중을 줄이고 싶다면 너무 많이 먹지 않되, 모든 종류의 음식, 채소, 과일 등을 먹어야 한다고 생각해요. 그리고 운동도 더 많이 해야 해요."
                    "#Person1#: 저도 그렇게 생각해요."
                    "\nSummary:\n"
                    "#Person1#은 #Person2#에게 #Person1#의 친구가 다이어트를 하다가 아파진 경험을 이야기한다. #Person2#는 사람들이 체중을 줄이려면 균형 잡힌 식단을 섭취하고 운동을 더 많이 해야 한다고 생각한다."
                    "\nExample 3:\n"
                    "Dialogue:\n"
                    "#Person1#: 크레이그, 너는 어떤 일을 하고 있니?"
                    "#Person2#: 나는 아직 학생이야."
                    "#Person1#: 어느 학교에 다니니?"
                    "#Person2#: 보스턴 대학교."
                    "#Person1#: 좋은 학교네. 무슨 공부를 하니?"
                    "#Person2#: 나는 영어, 수학, 역사를 공부하고 있어. 전공은 영어야."
                    "#Person1#: 얼마나 오래 영어를 공부해 왔니?"
                    "#Person2#: 6년 이상이야."
                    "#Person1#: 오랜 시간이네."
                    "#Person2#: 그래, 나는 고등학교 때부터 영어를 배우기 시작했어."
                    "#Person1#: 어쩐지 영어를 그렇게 잘하더라니."
                    "#Person2#: 사실, 그렇게 좋지는 않아. 읽는 건 할 수 있지만 말하기는 잘 못해. 연습할 기회가 많지 않았거든."
                    "#Person1#: 알겠어. 다른 사람들과 대화하는 것이 매우 중요하니까."
                    "#Person2#: 그래, 하지만 아직 여기서 많은 친구들을 사귀지 못했어."
                    "#Person1#: 오늘 밤에 내 아파트에서 파티를 열 건데. 와봐."
                    "#Person2#: 초대해줘서 고마워. 기꺼이 참석하겠어."
                    "\nSummary:\n"
                    "크레이그는 수년간의 영어 학습 경험이 있지만, 연습의 부족으로 인해 영어를 잘 말하지 못한다. #Person1#은 오늘 밤 파티에 크레이그를 초대한다."
                    "\nExample 4:"
                    "\nDialogue:\n"
                    "#Person1#: 아직 학생이신가요?"
                    "#Person2#: 네, 그런데 저는 다음 달에 상해 금융 무역 학교를 졸업할 예정입니다."
                    "#Person1#: 정말로 훌륭한 캐셔가 될 수 있을 거라고 확신하나요?"
                    "#Person2#: 확신합니다. 제 전공은 통계학이고, 제가 수강한 모든 과목에서 우수한 성적을 받았습니다."
                    "#Person1#: 그건 이론적인 기초에 불과합니다. 혹시 전에 큰 금액의 현금을 다뤄본 경험이 있나요?"
                    "#Person2#: 네, 아버지가 사업을 하셔서 저는 매 휴가마다 재무 부서에서 일할 기회가 있었습니다."
                    "#Person1#: 좋습니다. 당신은 좋은 캐셔가 될 것 같습니다."
                    "#Person2#: 감사합니다."
                    "\nSummary:\n"
                    "#Person2#는 #Person1#에게 우수한 성적을 얻었으며 실무 경험도 쌓았다고 말합니다. #Person1#는 #Person2#가 성공적인 캐셔가 될 것이라고 생각합니다."
                    )},
                {"role": "user", "content": f"Here is a dialogue in Korean:\n\n{dialogue}\n\nPlease summarize it in a way that feels like a machine-translated summary from English."}
                ],
            max_tokens=180,
            temperature=0.5
        )
        summary = response.choices[0].message.content.strip()
        print(f"Summary for index {index}: {summary}")  # 요약 결과 출력
        return summary
    except Exception as e:
        print(f"Error during summarization: {e}")
        return ""



SyntaxError: unexpected character after line continuation character (1250401869.py, line 62)

In [ ]:
def compute_metrics(predictions, references):
    rouge = Rouge()
    results = rouge.get_scores(predictions, references, avg=True)
    return {key: value["f"] for key, value in results.items()}


# 개발 데이터셋 평가
dev_df['predicted_summary'] = dev_df['dialogue'].apply(summarize_with_gpt)
metrics = compute_metrics(dev_df['predicted_summary'].tolist(), dev_df['summary'].tolist())
print("ROUGE Scores:", metrics)

Summary for index None: #Person1#은 #Person2#의 호흡이 힘들 때마다 가슴이 무겁게 느껴져서 #Person2#를 폐 전문의에게 보내 천식에 대한 검사를 받게 할 것이다.
Summary for index None: #Person1#은 지미에게 운동을 제안하지만, 지미는 다리가 아파서 거절한다. #Person1#은 그들의 운동 스케줄을 바꾸고 싶어하며, 지미는 동의한다.
Summary for index None: #Person1#은 더 이상 건강에 해로운 음식을 먹지 않기로 결정했고, #Person2#는 더 건강한 음식을 먹기 시작했다.
Summary for index None: #Person1#은 UFO를 믿지 않지만 #Person2#는 UFO를 믿는다. #Person2#는 #Person1#에게 꿈에서 UFO를 본다고 말하고, UFO가 지구에 오지 않는다고 말한다.
Summary for index None: #Person1#은 학교에 가지 않았지만, #Person2#는 영화 보러 갈 계획이다. #Person1#은 #Person2#에게 혼자 가라고 말하지만, #Person2#는 혼자 가고 싶지 않다.
Summary for index None: #Person1#은 #Person2#에게 담배를 끊으라고 요청하지만, #Person2#는 끊는 것이 어렵다고 생각한다. #Person1#은 #Person2#를 설득하고, #Person2#는 끊기로 결정한다.
Summary for index None: 셰리는 화이트 씨에게 나가기 전에 서명해야 할 것들을 요청하고, 화이트 씨는 그것들을 서명합니다.
Summary for index None: #Person1#은 카렌에게 해변에서의 주말을 묻는다. 카렌은 해변을 따라 조깅하고 배구를 했지만 수영은 하지 않았다. #Person1#은 그것이 편안해 보인다.
Summary for index None: #Person1#은 #Person2#에게 취미에 대해 묻는다. #Person2#는 야외에서 사진 찍는 것

In [ ]:
# dev_df에서 'summary'와 'predicted_summary' 열만 추출하여 CSV 파일로 저장
output_path = '/data/ephemeral/home/data/dev_predicted_summary.csv'

# 'summary'와 'predicted_summary' 열만 포함된 데이터프레임 생성
dev_predicted_df = dev_df[['summary','predicted_summary']]

# 데이터프레임을 CSV 파일로 저장
dev_predicted_df.to_csv(output_path, index=True)

# 저장 경로 출력
output_path

'/data/ephemeral/home/data/dev_predicted_summary.csv'

In [ ]:
# 요약 생성
submission_df = submission_df.merge(test_df[['fname', 'dialogue']], on='fname', how='left')
submission_df['summary'] = submission_df['dialogue'].apply(summarize_with_gpt)

# 결과 저장
submission_df = submission_df[['fname', 'summary']]  # 필요한 열만 남김
submission_df.to_csv('/data/ephemeral/home/GPT_finetuned_1128_3.csv', index=True)
print("Submission file created: GPT_finetuned_1128_3.csv")

Summary for index None: #Person1#은 더슨 씨에게 즉시 효력을 발휘하여 모든 직원에게 내부 메모를 배포하도록 요청합니다. 이 메모는 근무 시간 동안 즉시 메시지 사용을 금지합니다.
Summary for index None: #Person2#가 교통 체증 때문에 늦었다. #Person1#은 #Person2#에게 대중교통을 이용하고, 날씨가 좋을 때는 자전거를 타는 것을 제안한다.
Summary for index None: #Person1#은 케이트에게 마샤와 히어로가 이혼하려고 한다고 말한다. 케이트는 놀라워하고 그들이 잘 어울린다고 생각했다.
Summary for index None: #Person1#은 브라이언의 생일을 축하하고, 브라이언은 #Person1#이 기쁨을 느끼는 것을 바란다. 그들은 서로 칭찬한다.
Summary for index None: #Person1#과 #Person2#는 올림픽 공원을 방문하고 있습니다. #Person2#는 #Person1#에게 올림픽 공원의 정보를 알려줍니다.
Summary for index None: #Person1#은 회사에서의 일에 짜증이 나서 사업 계획서를 작성하고 투자자를 모집하려고 결정한다. #Person2#는 #Person1#에게 사업 계획서 작성에 대해 조언한다.
Summary for index None: #Person1#은 #Person2#가 수두에 걸렸다고 생각한다. #Person2#는 의사를 만나기 전에는 확신하지 못한다.
Summary for index None: #Person1#는 #Person2#에게 청구서를 주지만, #Person2#는 11월 20일에 이용한 세탁 서비스 비용이 다른 사람의 것이라고 말합니다. #Person1#는 죄송하게 생각하고 수정합니다.
Summary for index None: #Person1#은 스티븐에게 #Person1#의 아내가 #Person1#의 불륜을 발견했고 이혼하려고 한다고 말합니다.
Summary for index None: #Pe

In [ ]:
submission_df = submission_df[['fname', 'summary']]
submission_df.to_csv('/data/ephemeral/home/GPT_finetuned_output_last.csv', index=True)

In [ ]:
submission_df.to_csv('/data/ephemeral/home/GPT_output.csv', index=False)
print("Submission file created: GPT_output.csv")


Submission file created: GPT_output.csv


In [ ]:
# dev summary 길이 통계
dev_df['summary_length'] = dev_df['summary'].apply(len)
dev_summary_stats = dev_df['summary_length'].describe()
print(dev_summary_stats)
# 예측한 dev summary 길이 통계
dev_df['predicted_summary_length'] = dev_df['predicted_summary'].apply(len)
dev_predicted_summary_stats = dev_df['predicted_summary_length'].describe()
print(dev_predicted_summary_stats)